# SQL Options in Spark HW

Alirght let's apply what we learned in the lecture to a new dataset!

**But first!**

Let's start with Spark SQL. But first we need to create a Spark Session!

In [1]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# May take awhile locally
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


## Read in our DataFrame for this Notebook

For this notebook we will be using the Google Play Store csv file attached to this lecture. Let's go ahead and read it in. 

### About this dataset

Contains a list of Google Play Store Apps and info about the apps like the category, rating, reviews, size, etc. 

**Source:** https://www.kaggle.com/lava18/google-play-store-apps

In [3]:
path = 'Datasets/'

googlePlayStore = spark.read.csv(path+"googleplaystore.csv",header=True,inferSchema=True)

## First things first

Let's check out the first few lines of the dataframe to see what we are working with

In [9]:
googlePlayStore.toPandas().head(2)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up


As well as the schema to make sure all the column types were correctly infered

In [10]:
print(googlePlayStore.printSchema())

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


Looks like we need to edit some of the datatypes. We need to update Rating, Reviews and Price as integer (float for Rating) values for now, since the Size and Installs variables will need a bit more cleaning. Since we haven't been over this yet, I'm going to provide the code for you here so you can get a quick look at how it used (and how often we need it!).

**make sure to change the df name to whatever you named your df**

In [14]:
from pyspark.sql.types import IntegerType, FloatType
newdf = googlePlayStore.withColumn("Rating", googlePlayStore["Rating"].cast(FloatType())) \
            .withColumn("Reviews", googlePlayStore["Reviews"].cast(IntegerType())) \
            .withColumn("Price", googlePlayStore["Price"].cast(IntegerType()))
print(newdf.printSchema())
newdf.limit(5).toPandas()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)

None


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Looks like that worked! Great! Let's dig in. 

## 1. Create Tempview

Go ahead and create a tempview of the dataframe so we can work with it in spark sql.

In [15]:
# Create a temporary view of the dataframe
newdf.createOrReplaceTempView("tempview")

## 2. Select all apps with ratings above 4.1

Use your tempview to select all apps with ratings above 4.1

In [23]:
spark.sql("SELECT * FROM tempview WHERE Rating > 4.1").limit(5).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
1,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
2,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
3,Paper flowers instructions,ART_AND_DESIGN,4.4,167,5.6M,"50,000+",Free,0,Everyone,Art & Design,"March 26, 2017",1.0,2.3 and up
4,Garden Coloring Book,ART_AND_DESIGN,4.4,13791,33M,"1,000,000+",Free,0,Everyone,Art & Design,"September 20, 2017",2.9.2,3.0 and up


## 3. Now pass your results to an object 
(ie create a spark dataframe)

Select just the App and Rating column where the Category is in the Comic category and the Rating is above 4.5.

In [32]:
googlePlayStore.select("Category").distinct().show()

+-------------------+
|           Category|
+-------------------+
|             EVENTS|
|             COMICS|
|             SPORTS|
|            WEATHER|
|      VIDEO_PLAYERS|
|  AUTO_AND_VEHICLES|
|          PARENTING|
|      ENTERTAINMENT|
|    PERSONALIZATION|
| HEALTH_AND_FITNESS|
|   TRAVEL_AND_LOCAL|
|BOOKS_AND_REFERENCE|
|     FOOD_AND_DRINK|
|        PHOTOGRAPHY|
|           BUSINESS|
|             FAMILY|
|           SHOPPING|
|     HOUSE_AND_HOME|
|               GAME|
|               Face|
+-------------------+
only showing top 20 rows



In [31]:
spark.sql("SELECT App, Rating FROM tempview WHERE Category = 'COMICS' AND Rating > 4.5").limit(5).toPandas()

,App,Rating
0,Manga Master - Best manga & comic reader,4.6
1,GANMA! - All original stories free of charge f...,4.7
2,Röhrich Werner Soundboard,4.7
3,Unicorn Pokez - Color By Number,4.8
4,Manga - read Thai translation,4.6


## 4. Which category has the most cumulative reviews

Only select the one category with the most reivews. 

*Note: will require adding all the review together for each category*

In [64]:
spark.sql("SELECT Category, sum(Reviews) AS Total FROM tempview GROUP BY Category ORDER BY Total DESC").toPandas().head(1)

,Category,Total
0,GAME,1.585422e+09


## 5. Which App has the most reviews?

Display ONLY the top result

Include only the App column and the Reviews column.

In [66]:
spark.sql("SELECT App, Reviews AS Total FROM tempview ORDER BY Total DESC").limit(1).toPandas()

,App,Total
0,Facebook,78158306


## 5. Select all apps that contain the word 'dating' anywhere in the title

*Note: we did not cover this in the lecture. You'll have to use your SQL knowledge :) Google it if you need to.*

In [43]:
spark.sql("SELECT App FROM tempview WHERE App LIKE '%dating%'").toPandas()

,App
0,"Meet, chat & date. Free dating app - Chocolate..."
1,Friend Find: free chat + flirt dating app
2,Spine- The dating app
3,Princess Closet : Otome games free dating sim
4,happn – Local dating app


## 6. Use SQL Transformer to display how many free apps there are in this list

In [44]:
from pyspark.ml.feature import SQLTransformer

In [67]:
# Then we create an SQL call 
sqlTrans = SQLTransformer(
    statement="SELECT COUNT(App) FROM __THIS__ WHERE Type = 'Free'") 
# And use it to transform our df object
sqlTrans.transform(newdf).show(5)

+----------+
|count(App)|
+----------+
|     10037|
+----------+



## 7. What is the most popular Genre?

Which genre appears most often in the dataframe. Show only the top result.

In [54]:
sqlTrans = SQLTransformer(
    statement="SELECT Genres, COUNT(Genres) AS Total FROM __THIS__ GROUP BY Genres ORDER BY Total DESC") 
# And use it to transform our df object
sqlTrans.transform(newdf).limit(1)

+------+-----+
|Genres|Total|
+------+-----+
| Tools|  842|
+------+-----+



## 8. Select all the apps in the 'Tools' genre that have more than 100 reviews

In [62]:
sqlTrans = SQLTransformer(
    statement="SELECT * FROM __THIS__ WHERE Genres='Tools' AND Reviews>100") 
# And use it to transform our df object
sqlTrans.transform(newdf).toPandas()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Moto File Manager,TOOLS,4.1,38655,5.9M,"10,000,000+",Free,0.0,Everyone,Tools,"February 1, 2018",v3.7.93,5.0 and up
1,Google,TOOLS,4.4,8033493,Varies with device,"1,000,000,000+",Free,0.0,Everyone,Tools,"August 3, 2018",Varies with device,Varies with device
2,Google Translate,TOOLS,4.4,5745093,Varies with device,"500,000,000+",Free,0.0,Everyone,Tools,"August 4, 2018",Varies with device,Varies with device
3,Moto Display,TOOLS,4.2,18239,Varies with device,"10,000,000+",Free,0.0,Everyone,Tools,"August 6, 2018",Varies with device,Varies with device
4,Motorola Alert,TOOLS,4.2,24199,3.9M,"50,000,000+",Free,0.0,Everyone,Tools,"November 21, 2014",1.02.53,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
530,ChopAssistant,TOOLS,4.2,455,2.8M,"50,000+",Free,0.0,Everyone,Tools,"February 28, 2017",1.6,6.0 and up
531,Reindeer VPN - Proxy VPN,TOOLS,4.2,7339,4.0M,"100,000+",Free,0.0,Everyone,Tools,"May 10, 2018",1.74,4.1 and up
532,Inf VPN - Global Proxy & Unlimited Free WIFI VPN,TOOLS,4.7,61445,7.8M,"1,000,000+",Free,0.0,Everyone,Tools,"July 26, 2018",1.9.734,4.1 and up
533,FR Roster,TOOLS,4.1,174,12M,"5,000+",Free,0.0,Everyone,Tools,"July 30, 2018",6.04,4.4 and up


## That's all folks! Great job!